In [1]:
from __future__ import print_function
from __future__ import division
from importlib import reload

from nltk.tokenize.treebank import TreebankWordTokenizer
import collections
import nltk
import numpy as np
import glob
import math
import time

import pandas as pd
import tensorflow as tf
import os, sys, re, json, time, datetime, shutil
import random

# Helper libraries
from w266_common import utils, vocabulary
from w266_common import patched_numpy_io

from sentence import Sentence
from embedding import *
from training_set import TrainingSet 

/home/peterg/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/peterg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
KID_INAPPROPRIATE = 0
KID_APPROPRIATE = 1

labels = [KID_INAPPROPRIATE, KID_APPROPRIATE]


In [3]:
book_files = glob.glob('/home/peterg/mids-w266-final-project/final_proj/books/*.txt')
fourchan_files = glob.glob('/home/peterg/mids-w266-final-project/final_proj/4chan/*.txt')
single_4chan_file = ["/home/peterg/mids-w266-final-project/final_proj/4chan/4chan_s4s.txt",]
single_simple_wiki_file = ["/home/peterg/mids-w266-final-project/final_proj/wiki/simple.txt",]
single_normal_wiki_file = ["/home/peterg/mids-w266-final-project/final_proj/wiki/normal.txt",]

merged_pos_files = ["/home/peterg/mids-w266-final-project/final_proj/merged/merged_pos.txt",]
merged_neg_files = ["/home/peterg/mids-w266-final-project/final_proj/merged/merged_neg.txt",]

big_merged_pos_files = ["/home/peterg/mids-w266-final-project/final_proj/merged/merged_pos_100k.txt",]
big_merged_neg_files = ["/home/peterg/mids-w266-final-project/final_proj/merged/merged_neg_100k.txt",]

In [4]:
NUM_EXAMPLES = 20000
# NUM_EXAMPLES = 100000
MAX_VOCAB = 70000
PAD_LEN = 40
tokenizer = TreebankWordTokenizer()

test_frac = 0.2
train_frac = 0.8

In [5]:
comment_set = TrainingSet(book_files, single_4chan_file, tokenizer, NUM_EXAMPLES)
wiki_set = TrainingSet(single_simple_wiki_file, single_normal_wiki_file, tokenizer, NUM_EXAMPLES)
small_merge_set = TrainingSet(merged_pos_files, merged_neg_files, tokenizer, NUM_EXAMPLES)
merge_set = TrainingSet(big_merged_pos_files, big_merged_neg_files, tokenizer, NUM_EXAMPLES)

training_sets = [comment_set, wiki_set, small_merge_set, merge_set]

In [6]:
for ts in training_sets:
    ts.generate_vocab(MAX_VOCAB)
    ts.prep_sents(PAD_LEN)
    ts.divide_test_train(test_frac, train_frac)
    ts.prep_sets()

In [7]:
test_arr = [("ass and titties", KID_INAPPROPRIATE),
("fuck the police", KID_INAPPROPRIATE),
("cats are nice", KID_APPROPRIATE),
("one day I saw a horse on a hill and I liked it", KID_APPROPRIATE),
("mrs. tayler is a whore", KID_INAPPROPRIATE),
("I'd like to take you someplace nice and quiet", KID_INAPPROPRIATE),
("You gay ? Proceeds to chase him that's how bullying works", KID_INAPPROPRIATE),
("transfer responsibility dad temperature earn voter impossible radiation.", KID_APPROPRIATE),
("JADE CHYNOWETH AND JOSH KILLACKY LOVETEAM PLS", KID_APPROPRIATE),
("Lol, looks like the same generic crap you come to expect from these types of shows.  Guess it doesn't get old for some people.", KID_INAPPROPRIATE),
("Disney sold the rights to this?", KID_APPROPRIATE),
("Only step up 1 and 2 were good. All of the other ones have just been corny.", KID_APPROPRIATE),
("i rather not call it step up, its another level with different camera techniques and its so Channing Tatum style", KID_APPROPRIATE),
("I love the series its very good, im waiting for 2 one", KID_APPROPRIATE),
("GREAT", KID_APPROPRIATE),
("Job negotiate set alternative little introduction apparent crazy proper used care free.", KID_APPROPRIATE),
("Oxygen identify member dependent translate else card might handful.", KID_APPROPRIATE),
("Release accompany pole general something widely fly cup detective personnel.", KID_APPROPRIATE),
("Silly largely obstacle warrior charge flavor diabetes medal.", KID_APPROPRIATE),
("We need moose back", KID_APPROPRIATE),
("Exciting time to be alive bro watching our Prez Trump vs. the DeepState/Swamp that's the deal profs to him.", KID_APPROPRIATE),
("Idk if I want to watch it or not : ^/", KID_APPROPRIATE),
("Maximum testing blanket absolutely shock until actress sex liability.", KID_INAPPROPRIATE),
("3 was the best.", KID_APPROPRIATE),
("I knew they would have a all men gay dance group", KID_INAPPROPRIATE),
("Same old garbage; I'm sick of it !", KID_APPROPRIATE),
("GAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAYYYYYYYYYYYYYYYYYYYYYYYYY",KID_INAPPROPRIATE),
("Millenials = The worst generation",KID_APPROPRIATE),
("that opening statement is the most overused sentence i've seen",KID_APPROPRIATE),
("Pan spokesman via guard campaign characteristic movement expert attend garage.",KID_APPROPRIATE),
("ayyy https://youtu.be/wZwyQZDfyHY",KID_APPROPRIATE),
("WELP....the end of the step up franchise met a very disappointing end. Jesus this looks worse than most dumpster fires Ive seen",KID_APPROPRIATE),
("blacks that look white and wiggas.  yeah, looks great. (not!)",KID_INAPPROPRIATE),
("Rich And Diverse artistry- Advocating White Genocide. How is enrichment going on in South Africa?",KID_INAPPROPRIATE),
("step up <3",KID_APPROPRIATE),
("When is episode 5 coming out",KID_APPROPRIATE),
("This series is actually pretty good...watched it and was so happy I gave it a shot; I loved it! The storyline is actually stronger than I expected, and especially where they left off this first season, I'm so excited for what's to come; I hope they keep up the good work and produce even more (authentic stuff next season.",KID_APPROPRIATE),
("https://vk.com/club162181675 вступайте",KID_APPROPRIATE),
("lunch argument across expansion free govern healthy afternoon.",KID_APPROPRIATE),
("Modern day version of Fame!",KID_APPROPRIATE),
("Neyo!",KID_APPROPRIATE),
("I wasn't expecting this from YouTube but it's really good",KID_APPROPRIATE),
("The",KID_APPROPRIATE),
("what garbage i this want to be step up is more like stomp the yard",KID_APPROPRIATE),
("if you're actually thinking of watching this show... do it. i recommend it. it's so much more than what we ever got to see from step up or any dance movies at all. so much diversity & so many topics it's talking about. i was surprised at how much i actually enjoyed the show & how bad i need a (second season. just give it a try since the first 4 episodes are for free on here, i don't think y'all are gonna regret it",KID_APPROPRIATE),
("Aren’t u guys releasing more seasons and episodes??? we still need more of ‘em",KID_APPROPRIATE),
("Have Chris brown in the step up movies it’ll be dope",KID_APPROPRIATE),
("http://hikmatblogs.blogspot.com/2018/02/10-ways-to-improve-your-finances-today.html",KID_APPROPRIATE),
("No Moose no Step Up",KID_APPROPRIATE),
("Let me guess?? Suburban girl trying to prove her gangsta and most of the black men are gay.... WHACK!!!",KID_INAPPROPRIATE),
("What, no moose cameo ?",KID_APPROPRIATE),
("trash.",KID_APPROPRIATE),
("Why does everything have to be so gay  won't be watching this sick of all our black men in tv series being gay smh",KID_INAPPROPRIATE),
("Who's that girl on the trailer cover?",KID_APPROPRIATE),
("Atlanta is a n!66er filled cesspool.. i can't figure out why parents stopped teaching their children white from wrong.. why today's youth wants grey babies.. why ruin family pictures with those half n!66e kinky haired flat nosed kids..  why these fathers aren't teaching their daughters to stay off the chimpdicks.. i mean, for real, how low does your self esteem have to be to want to share your beautiful young white body with these savage nigloyds... i'm sorry your fathers have failed you girls.. I taught mine white from wrong.. my grandkids will be bright white with blonde hair and blue eyes.. not gray with kinky rugs and flat noses.. just the idea of beautiful young white girls with sweaty, greasy n!66ers up on them just makes me wanna puke.. parents.. it's time to start teaching some basic family values again.. not the kardashian kind where they just keep getting impregnated by stray chimps.. it's just disgusting... let's clean up and keep the white blood lines free from contaminants..  you young white girls... just say no.. find a decent white boy to make babies with.. you can't tell me that a n!66er is all you can get..  well, maybe the fat ugly girls.. but even you can do better.. find a fat ugly white guy..  give him that poontang .. not these foot stomping, drum banging monkeys .. talentless rappers...  all wanna pretend they are in the music business.. rap isn't music.. any tar baby can get a drum machine and call it a beat lab.. then they steal a macbook from a white girl.. and start their career as the white girl's baby daddy.. she says he's a musician.. but he's really just another worthless n166er  he will keep her on welfare.. just a waste.. february is black history month.. the other 11 is caucasian (history months.. we will celebrate by taking our women back..",KID_INAPPROPRIATE),
("i run them streetsXD",KID_APPROPRIATE),
("my answer is....hell noXD",KID_APPROPRIATE),
("Sail star diamond rate working love pond size that monument prevention celebrate.",KID_APPROPRIATE),
("So is Kevin Bacon they're Jesus or something?",KID_APPROPRIATE),
("Can someone please give me the name of the girl on the thumbnail please ?????",KID_APPROPRIATE),
("I would rather be prison raped than watch 5seconds of this garbage",KID_INAPPROPRIATE),
("Prosecution tool before endless visit dump shake sake remarkable hurt safe public.",KID_APPROPRIATE),
("total grief slave esyvbp question fourth fun basic fly sacrifice reply.",KID_INAPPROPRIATE),
("As she named the Empress, Anna Pávlovna’s face suddenly assumed an expression of profound and sincere devotion and respect mingled with sadness, and this occurred every time she mentioned her illustrious patroness. ", KID_INAPPROPRIATE),
("She added that Her Majesty had deigned to show Baron Funke beaucoup d’estime, and again her face clouded over with sadness.", KID_INAPPROPRIATE),
("The prince was silent and looked indifferent. ", KID_APPROPRIATE),
("But, with the womanly and courtierlike quickness and tact habitual to her, Anna Pávlovna wished both to rebuke him (for daring to speak as he had done of a ma ", KID_INAPPROPRIATE),
("recommended to the Empress) and at the same time to console him, so she said ", KID_INAPPROPRIATE),
("Such lame shit", KID_INAPPROPRIATE),
("Who the fuck is IcePoseidon and why do we have to watch him be stupid", KID_INAPPROPRIATE),
("Legends say he's still about to lose his job", KID_APPROPRIATE),
("WTF", KID_INAPPROPRIATE),
("Classic Greeks", KID_APPROPRIATE),
("8:27 de guy was jerking his dick", KID_INAPPROPRIATE),
("Roses are red violets are blue I just got click baited and so did you", KID_APPROPRIATE),
("Worst ad campaign I have ever seen. And this is after the 2016 elections", KID_APPROPRIATE),
("bleeps words but shows a 45 second clip of a guy masturbating. Real high standards we have here", KID_INAPPROPRIATE),
("ok Ice posiden ACIDENTLY this many times is fake", KID_APPROPRIATE),
("It seems like Ice Poseidon can’t get his shit together and stream accordingly to his schedule since he’s got the most accidental live starts. Never liked him and seeing his fuck ups just makes himself look bad", KID_INAPPROPRIATE),
("did he lose his job", KID_APPROPRIATE),
("Hahahahaha", KID_APPROPRIATE),
("greek, no one will ever suck your dick", KID_INAPPROPRIATE),
("I didn't understand most of these", KID_APPROPRIATE),
("4:04 its not accidentally ?", KID_APPROPRIATE),
("why is there 3k dislikes", KID_APPROPRIATE),
("That isn’t a dildo, that’s a motion controller.....", KID_INAPPROPRIATE),
("A GUITAR.. she tried so hard. I probably would’ve acted the same way lol", KID_APPROPRIATE),
("Why does people watch streaming people. Fucking dumb", KID_INAPPROPRIATE),
("7:52 that was merchant saying I'm to busy running hes on youtube", KID_APPROPRIATE),
("4:20 what did he do", KID_APPROPRIATE),
("who's the last girl", KID_APPROPRIATE),
("Did he lose his job though", KID_APPROPRIATE),
("Always thumbs down to these videos. Don't know why they're recommended to me. Cancerous, toxic ass puss, all of it", KID_INAPPROPRIATE),
("That's mad, I went to school with the guy hosting the RS stream at 7:30. Sat next to him in GCSE IT", KID_APPROPRIATE),
("I knew he did a lot of RS videos but not stuff like this", KID_APPROPRIATE),
("Did he lose his job", KID_APPROPRIATE),
("What the fuck was said to the girl at the end", KID_INAPPROPRIATE),
("Where the fuck does Ninga forget", KID_INAPPROPRIATE),
("Whats that under your bed? Umm karaoke mic", KID_APPROPRIATE),
("that was my frend", KID_APPROPRIATE),
("im playing", KID_APPROPRIATE),
("What’s that under your bed.  It’s a fucking jumprope", KID_INAPPROPRIATE),
("What was that thing under they girl’s bed", KID_APPROPRIATE),
("Intro song please", KID_APPROPRIATE),
("Man at 8:00 had a terrible brain freeze", KID_APPROPRIATE),
("Wait I don't get it, why are they speaking to a chat if it's not live.", KID_APPROPRIATE),
("Did he lose his job or nah", KID_APPROPRIATE),
("At 1:30 what was under her bed", KID_APPROPRIATE),
("Lol!! He jacked off on the fucking twitch can and noticed and just fucking 😂😂😂😂😂", KID_INAPPROPRIATE),
("What happened with the guy saying he gonna lose his job? WhAt was the big deal", KID_APPROPRIATE),
("CAN SOMEONE TELL ME WHAT RUNESCAPE LEAKED LOL", KID_APPROPRIATE),
("About 70% weren't accidental live streams", KID_APPROPRIATE),
("Did he lose his job", KID_APPROPRIATE),
("Could someone please tell me the BGM at around 2:13? Please, really appreciate it", KID_APPROPRIATE),
("Super", KID_APPROPRIATE),
("Who's running this fucking channel? Get rid of these titty streamer thumbnails, dude. Real gross shit especially with this latest one", KID_INAPPROPRIATE),
("Fuck em", KID_INAPPROPRIATE),
("My God it's called a bandaid cover the camera when not in use", KID_APPROPRIATE),
("Accidental my ass", KID_INAPPROPRIATE)]

In [8]:
import models;

# ts = comment_set
ts = wiki_set

def train_and_test_bow(ts):
    start_time = time.time()
    x, ns, y, ext_y = ts.trains
    
    test_set = TrainingSet()

    test_set.load_test_arr(test_arr,tokenizer)
    test_set.set_vocab(ts.vocab)
    test_set.prep_sents(PAD_LEN)
    test_set.prep_sets()

    # Training params
    batch_size = 64
    model_params = dict(V=ts.vocab, embed_dim=70, hidden_dims=[75,50,25], num_classes=len(labels),
                        encoder_type='bow',
                        lr=0.1, optimizer='adagrad', beta=0.01)
    model_fn = models.classifier_model_fn

    # training
    total_batches = 0
    total_examples = 0
    total_loss = 0
    loss_ema = np.log(2)  # track exponential-moving-average of loss
    ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
    with tf.Graph().as_default(), tf.Session() as sess:
        ##
        # Construct the graph here. No session.run calls - just wiring up Tensors.
        ##
        # Add placeholders so we can feed in data.
        x_ph_  = tf.placeholder(tf.int32, shape=[None, x.shape[1]])  # [batch_size, max_len]
        ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
        y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]

        # Construct the graph using model_fn
        features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
        estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                                  params=model_params)
        loss_     = estimator_spec.loss
        train_op_ = estimator_spec.train_op

        ##
        # Done constructing the graph, now we can make session.run calls.
        ##
        sess.run(tf.global_variables_initializer())


    checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
    if os.path.isdir(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)
    # Write vocabulary to file, so TensorBoard can label embeddings.
    # creates checkpoint_dir/projector_config.pbtxt and checkpoint_dir/metadata.tsv
    ts.vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

    model = tf.estimator.Estimator(model_fn=models.classifier_model_fn, 
                                   params=model_params,
                                   model_dir=checkpoint_dir)
    
    print("")
    print("To view training (once it starts), run:\n")
    print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
    print("\nThen in your browser, open: http://localhost:6006")
    
    # Training params, just used in this cell for the input_fn-s
    train_params = dict(batch_size=128, total_epochs=20, eval_every=10)
    assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

    # Construct and train the model, saving checkpoints to the directory above.
    # Input function for training set batches
    # Do 'eval_every' epochs at once, followed by evaluating on the dev set.
    # NOTE: use patch_numpy_io.numpy_input_fn instead of tf.estimator.inputs.numpy_input_fn
    train_input_fn = patched_numpy_io.numpy_input_fn(
                        x={"ids": x, "ns": ns}, y=y,
                        batch_size=train_params['batch_size'], 
                        num_epochs=train_params['eval_every'], shuffle=True, seed=42
                     )
    
    # Input function for dev set batches. As above, but:
    # - Don't randomize order
    # - Iterate exactly once (one epoch)
    dev_x, dev_ns, dev_y, dev_y_ext = ts.devs
    dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                        x={"ids": dev_x, "ns": dev_ns}, y=dev_y,
                        batch_size=128, num_epochs=1, shuffle=False
                    )

    for _ in range(train_params['total_epochs'] // train_params['eval_every']):
        # Train for a few epochs, then evaluate on dev
        model.train(input_fn=train_input_fn)
        eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")
        
    sd_test_x, sd_test_ns, sd_test_y, _ = ts.sds
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
                        x={"ids": sd_test_x, "ns": sd_test_ns}, y=sd_test_y,
                        batch_size=128, num_epochs=1, shuffle=False
                    )

    same_domain_eval = model.evaluate(input_fn=test_input_fn, name="eval")


    print("Accuracy on same domain test set: {:.02%}".format(same_domain_eval['accuracy']))

    
    test_x, test_ns, test_y, test_y_ext = test_set.ext_tests
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
                        x={"ids": test_x, "ns": test_ns}, y=test_y,
                        batch_size=128, num_epochs=1, shuffle=False
                    )

    new_domain_eval = model.evaluate(input_fn=test_input_fn, name="eval")
    print("Accuracy on generalized test set: {:.02%}".format(new_domain_eval['accuracy']))
    
    from sklearn.metrics import accuracy_score
    predictions = list(model.predict(test_input_fn))  # list of dicts
    y_pred = [p['max'] for p in predictions]

    end_time = time.time()
    total_time = end_time - start_time
    return same_domain_eval, new_domain_eval, y_pred, total_time

In [9]:
# bow_sd_comment, bow_nd_comment, bow_comment_preds, bct = train_and_test_bow(comment_set)
# print("comments done")
# bow_sd_wiki, bow_nd_wiki, bow_wiki_preds, bwt          = train_and_test_bow(wiki_set)
# print("wiki done")
# bow_sd_merge, bow_nd_merge, bow_merge_preds, bmt       = train_and_test_bow(merge_set)
# print("merged done")

In [10]:
# print("bow_sd_comment", bow_sd_comment)
# print("bow_nd_comment", bow_nd_comment)
# print(bct)

# print("\nbow_sd_wiki", bow_sd_wiki)
# print("bow_nd_wiki", bow_nd_wiki)
# print(bwt)

# print("\nbow_sd_merge",bow_sd_merge)
# print("bow_nd_merge", bow_nd_merge)
# print(bmt)

# CNN

In [11]:
EMBED_DIM = 90
CELL_SIZE = 40
RNN_NUM = 3
learning_rate = 0
dropout_keep_prob = 0
lam = 1
train_embedding = False
embedding_path = ""
model_dir = "runs"
summaries_dir = "summary"

In [12]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime

from text_cnn import TextCNN
from tensorflow.contrib import learn

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 128, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 10)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=128
CHECKPOINT_EVERY=100
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NUM_CHECKPOINTS=5
NUM_EPOCHS=10
NUM_FILTERS=128



In [13]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [14]:
def train_and_test_cnn(ts, use_embedding = False):
    start_time = time.time()
    x, _, y, y_ext = ts.trains
    dev_x, _, dev_y, dev_y_ext = ts.devs
    test_set = TrainingSet()

    test_set.load_test_arr(test_arr,tokenizer)
    test_set.set_vocab(ts.vocab)
    test_set.prep_sents(PAD_LEN)
    test_set.prep_sets()
    
    test_x, test_ns, test_y, test_y_ext = test_set.ext_tests
    sd_test_x, _, sd_test_y, sd_test_y_ext = ts.sds
    
    # Training
    # ==================================================
    last_path = ""
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        if use_embedding:
            embedding_path = "/home/peterg/word2vec-GoogleNews-vectors/GoogleNews-vectors-negative300-SLIM.bin"
        else:
            embedding_path = ""
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x.shape[1],
                num_classes=len(labels),
                vocab=ts.vocab,
                embedding_size=FLAGS.embedding_dim,
                filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters=FLAGS.num_filters,
                l2_reg_lambda=FLAGS.l2_reg_lambda,
                embedding_path=embedding_path)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Write vocabulary
            ts.vocab.write_flat_file(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                if step % 100 == 0:
                    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            # Generate batches
            batches = batch_iter(
                list(zip(x, y_ext)), FLAGS.batch_size, FLAGS.num_epochs)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % FLAGS.evaluate_every == 0:
                    print("\nEvaluation:")
                    dev_step(dev_x, dev_y_ext, writer=dev_summary_writer)
                    print("")
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))
                    last_path = path
    def test(test_x, test_y):
    #     checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
        checkpoint_file = path
        graph = tf.Graph()
        with graph.as_default():
            session_conf = tf.ConfigProto(
              allow_soft_placement=FLAGS.allow_soft_placement,
              log_device_placement=FLAGS.log_device_placement)
            sess = tf.Session(config=session_conf)
            with sess.as_default():
                # Load the saved meta graph and restore variables
                saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
                saver.restore(sess, checkpoint_file)

                # Get the placeholders from the graph by name
                input_x = graph.get_operation_by_name("input_x").outputs[0]
                # input_y = graph.get_operation_by_name("input_y").outputs[0]
                dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

                # Tensors we want to evaluate
                predictions = graph.get_operation_by_name("output/predictions").outputs[0]

                # Generate batches for one epoch
                batches = batch_iter(list(test_x), FLAGS.batch_size, 1, shuffle=False)

                # Collect the predictions here
                all_predictions = []

                for x_test_batch in batches:
                    batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
                    all_predictions = np.concatenate([all_predictions, batch_predictions])

        # Print accuracy if y_test is defined
        if test_y is not None:
            correct_predictions = float(sum(all_predictions == test_y))
            print("Total number of test examples: {}".format(len(test_y)))
            print("Accuracy: {:g}".format(correct_predictions/float(len(test_y))))
            return correct_predictions/float(len(test_y)), all_predictions
    same_domain, sd_preds = test(sd_test_x, sd_test_y)
    new_domain, nd_preds = test(test_x, test_y)
    end_time = time.time()
    total_time = end_time - start_time
    return same_domain, new_domain, nd_preds, total_time

In [15]:
cnn_sd_comment, cnn_nd_comment, cnn_preds_comment, cct = train_and_test_cnn(comment_set, True)
print("comments done")
cnn_sd_wiki, cnn_nd_wiki, cnn_preds_wiki, cwt          = train_and_test_cnn(wiki_set, True)
print("wiki done")
cnn_sd_merge, cnn_nd_merge, cnn_preds_merge, cmt       = train_and_test_cnn(merge_set, True)
print("merge done")

TypeError: Value passed to parameter 'input' has DataType float64 not in list of allowed values: float16, float32

In [ ]:
print("sd comment Accuracy: {:g}".format(cnn_sd_comment))
print("nd comment Accuracy: {:g}".format(cnn_nd_comment))
print(cct)

print("\n")
print("sd wiki Accuracy: {:g}".format(cnn_sd_wiki))
print("nd wiki Accuracy: {:g}".format(cnn_nd_wiki))
print(cwt)
print("\n")
print("sd merge Accuracy: {:g}".format(cnn_sd_merge))
print("nd merge Accuracy: {:g}".format(cnn_nd_merge))
print(cmt)

-- https://www.tensorflow.org/tutorials/text_classification_with_tf_hub
-- https://github.com/dennybritz/cnn-text-classification-tf
-- With word2vec
-- https://github.com/cahya-wirawan/cnn-text-classification-tf

In [ ]:
small_cnn_sd_merge, small_cnn_nd_merge, small_cnn_preds_merge, cmt       = train_and_test_cnn(small_merge_set, True)

In [ ]:
print("small sd merge Accuracy: {:g}".format(small_cnn_sd_merge))
print("small nd merge Accuracy: {:g}".format(small_cnn_nd_merge))
print(cmt)

In [ ]:
!which python